In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
directory = "./Logs/"
save_path = "./Saved/"

from load_logs import *
from evaluation import *
from features import *
from visualize import *

logs, log_labels = read_logs_and_labels("./Saved/logs.txt", "./Saved/labels.txt")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import os
import pickle
import sys
import random
import hdbscan
import itertools

from bregclus.models import BregmanHard
from bregclus.divergences import euclidean
from multiprocessing import get_context
from sklearn.cluster import Birch
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_samples
from scipy import sparse
from multiprocessing import Pool
from math import sqrt
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [3]:
#Time to define necessary functions
#Since this is based on a knn idea we will need to calculate euclidean distance

In [4]:
#Nearest neighbour - gets x number of nearest nodes
def nearest(x, y_row, neighbors, labels): #specify number of neighbours to return
    distances = np.linalg.norm(x - y_row, axis=1)#numpy array of distances
    size, y = np.shape(x)
    itera = np.arange(0,size).reshape(-1,1)
    for i in range(len(labels)):
        distances = np.c_[distances,labels[i]]
    distances = np.c_[distances,itera]
    distances = distances[np.argsort(distances[:, 0])]
    neighborsList, neighborsLabels, iteraList = list(), list(), list()
    for i in range(neighbors): #27093
        neighborsList.append(distances[i][0])
        neighborsLabels.append(distances[i][1:len(distances[i])-1])
        iteraList.append(distances[i][len(distances[i])-1])
    return neighborsList,neighborsLabels,iteraList

In [5]:
def lookUp(labels, lookup, which):
    arr = np.array(labels)
    x,y = np.shape(arr)
    indices = np.argwhere(arr[which] == lookup)
    return indices

In [6]:
def firstTrue(boolArr):
    for i in range(len(boolArr)):
        if boolArr[i] == True:
            return i

In [7]:
def recluster(x,y,silVal, itera): #Recluster based on silhouette coefficient (i.e whichever node here has a higher coefficient)
    print(itera)
    if silVal[int(itera[0])] > silVal[int(itera[1])]:
        return x
    else:
        return y    

In [8]:
def isEqual(x,y):
    if x == y:
        return True
    else:
        return False

In [9]:
#Current and nearest are same cluster then true otherwise false
def checkClus(x,y):
    boolArr = list(map(isEqual, x, y))
    return boolArr

In [10]:
def consensus(x,y,labels,silVal,itera,silVal1): #form boolean array from checkClus, if majority are true then the node should be in the cluster, otherwise not
    boolArr=checkClus(x,y)
    trueCount, total,first = sum(boolArr), len(boolArr), firstTrue(boolArr)
    if (trueCount)/(total) > 0.5:
        #pass vote of node being in cluster
        if (boolArr[0] == False): #when consensus is passed but what our clustering is based on does not agree
            print("did not fully pass")
            ind1 = lookUp(labels,x[first],first)#need to sort this out such that we determine if the nearest nodes value is better
            ind2 = lookUp(labels,x[0],0)#or if the current nodes value is better
            ind3 = lookUp(labels,y[0],0)
            firstValList = np.intersect1d(ind1, ind2, assume_unique=False, return_indices=False)
            x1 = np.shape(firstValList)
            secondValList = np.intersect1d(ind1, ind3, assume_unique=False, return_indices=False)
            x2 = np.shape(secondValList)
            if x1 > x2: #determine which node classification is agreed on more
                print("replacing "+ str(x[0]) +" with "+ str(x[0]) + " at "+str(itera))
                return True, x[0]
            else:
                print("replacing "+ str(x[0]) +" with "+ str(y[0])+ " at "+str(itera))
                return True, y[0]
            #return True, recluster(x[0],y[0],silVal1,itera)
        return True, x[0]
    else:
        print("failed!")
        
        #pass the false vote - node is not in cluster
        return False, recluster(x[0],y[0],silVal1,itera)

In [11]:
def comprehension(X,labels,x,itera,silVal):
    silValed = silVal
    lb = list()
    lb.append(labels[0][itera])
    lb.append(labels[1][itera])
    lb.append(labels[2][itera])
    labelsed = (np.reshape(np.array(labels),(-1,3))).tolist()
    labs = labels
    it = list()
    if itera > 0:
        X = np.delete(X,[0,itera-1], axis = 0)
        labels = np.delete(labels,[0,itera-1], axis = 1)
        silValed = np.delete(silVal,[0,itera-1], axis = 0)
    near, tempLab,inds = nearest(X, x, 2, labels)
    it.append(itera)
    it.append(inds[1])
    boolArr1,res1 = consensus(lb,tempLab[1],labs,silValed,it,silVal)
    if boolArr1 == True:
        return int(res1) #need to sort this out such that we assign a valid cluster number
    else: #recluster somehow
        print("Failed - reclustering")
        return int(res1)

    

In [12]:
def takeConsensus(X, labels,silVal):
    x,y = np.shape(X)
    i = list(range(0,x))
    tasks = [*zip(itertools.repeat(X,x),itertools.repeat(labels,x), X,i,itertools.repeat(silVal,x))]
    print("running consensus")
    #p = Pool(6)#Multiprocessing to speed up run time
    with Pool() as p:
    #conList = [comprehension(X, labels, x) for x in X]
        conList = p.starmap(comprehension, tasks)
    #conList = starmap(comprehension, tasks)
    #for i in range(len(X)):
        #boolArr = list()
        #near, tempLab = nearest(X_, X[i], 3, labels) #do not repeat values in proximity
        #X_ = np.delete(X, i, axis = 0) #This will exclude the current node from our next nearest calculations
        #boolArr1 = checkClus(tempLab[0],tempLab[1])
       # boolArr2 = checkClus(tempLab[0],tempLab[2])
        #if consensus(boolArr1):
          #  conList.append(tempLab[0][0]) #need to sort this out such that we assign a valid cluster number
        #elif consensus(boolArr2): #here we check its second nearest neighbor before reclustering
         #   conList.append(tempLab[1][0])
       # else: #recluster somehow
           # conList.append(tempLab[0][0])
        #print(conList)
    return conList

In [13]:
labels_ = log_labels
            
X = get_features(logs, 2, 70)
X = X.toarray()

idxs = np.where(np.all(X == 0, axis=1))

X = np.delete(X, idxs, axis=0)
labels_ = np.delete(labels_, idxs)
## K Means Labels
kmeans = KMeans(init="k-means++", n_clusters=10,random_state=0).fit(X)
kLabels = kmeans.labels_

## DBSCAN Labels
DBmodel = DBSCAN(eps=0.4,min_samples=200,algorithm="auto",metric="cosine")
DBmodel.fit(X)
dbLabels = DBmodel.labels_

## Birch Label
birchmodel = Birch(n_clusters=10, branching_factor=100, threshold=0.5, compute_labels=True)
Y = birchmodel.fit_transform(X)
birchLabels = birchmodel.labels_

#HDBScan Labels
model = hdbscan.HDBSCAN(min_cluster_size=500,min_samples=100,cluster_selection_epsilon=0.5,
                                        metric="euclidean",algorithm='best')
model.fit(X)
hdbLabels = model.labels_


In [40]:
#Bregman Labels
bhc = BregmanHard(n_clusters=10, divergence=euclidean)
bhc.fit(X)
bLabels = bhc.predict(X)

In [41]:
silVal = silhouette_samples(X, kLabels)

In [42]:
#testing
labels = list()
labels.append(kLabels)
labels.append(hdbLabels)
labels.append(bLabels)

#x,y,z = nearest(X, X[0], 2, labels)
#print(x)
#print(y)
#print(z)
#boolArr = checkClus(y[0],y[1])
#print(boolArr)

#print(consensus(y[0],y[1],labels,silVal))
#x,y = np.shape(X)
#print(x)

In [43]:
print(np.shape(labels))
print(np.shape(X))

(3, 27093)
(27093, 1832)


In [ ]:
itertools.repeat(silVal, x)

In [ ]:
print(np.shape(silVal))
print(np.shape(X))
print(np.shape(labels))

In [ ]:
print(recluster(labels[0][26903],labels[0][6036],silVal, [26903,6036]))
print(silVal[26903])
print(silVal[6036])

In [ ]:
import worker

if __name__ == '__main__':
    x,y = np.shape(X)
    i = list(range(0,x))
    tasks = [*zip(itertools.repeat(X, x),itertools.repeat(labels, x), X, i, itertools.repeat(silVal, x))]
    p = Pool(4)
    cLabels = p.starmap(worker.comprehension, tasks)
    print(cLabels)
    p.close()
    p.join()

In [ ]:
print(kLabels[11765])
print(cLabels[11765])
print(kLabels - cLabels)
print(np.shape(np.nonzero(kLabels - cLabels)))
#12 nodes changed with db, hdb (in order) - hdb is a better option
#11 nodes changed with hdb, birch

In [19]:
print(set(cLabels) - set(kLabels))

set()


In [ ]:
#print((np.reshape(np.array(labels),(-1,3))).tolist())
x = list()
x.append(labels[0][3144])
x.append(labels[1][3144])
x.append(labels[2][3144])
y = list()
y.append(labels[0][11981])
y.append(labels[1][11981])
y.append(labels[2][11981])
recluster(labels[0][3144],labels[0][11981],silVal, [3144, 11981.0])

In [ ]:
print(comprehension(X,labels,X[3144],3144,silVal)) #did not fully pass
#[3590.0, 1393.0]
#Succesful recluster
#did not fully pass
#[1002.0, 19211.0]
#Succesful recluster
#failed!
#[4676.0, 11270.0]
#Succesful recluster
#Failed - reclustering
#did not fully pass
#[3144.0, 1403.0]
#Succesful recluster
#did not fully pass
#[8627.0, 24408.0]

#fixed issue where passing wrong label list
print(str(labels[0][4676]) + " labels")

In [ ]:
plot_clusters("Consensus Bigram Clustering using UMAP", X, cLabels)

In [ ]:
results = evaluate_clustering('Consensus Bigram Clustering', X, labels_, cLabels)
print(results)

In [ ]:
labels_ = log_labels
            
X = get_features(logs, 3, 90)
X = X.toarray()

idxs = np.where(np.all(X == 0, axis=1))

X = np.delete(X, idxs, axis=0)
labels_ = np.delete(labels_, idxs)
## K Means Labels
kmeans = KMeans(init="k-means++", n_clusters=10,random_state=0).fit(X)
kLabels = kmeans.labels_

## DBSCAN Labels
DBmodel = DBSCAN(eps=0.4,min_samples=200,algorithm="auto",metric="cosine")
DBmodel.fit(X)
dbLabels = DBmodel.labels_

## Birch Label
birchmodel = Birch(n_clusters=10, branching_factor=100, threshold=0.5, compute_labels=True)
Y = birchmodel.fit_transform(X)
birchLabels = birchmodel.labels_

In [ ]:
labels = list()
labels.append(kLabels)
labels.append(dbLabels)
labels.append(birchLabels)

In [ ]:
import worker

if __name__ == '__main__':
    x,y = np.shape(X)
    i = list(range(0,x))
    tasks = [*zip(itertools.repeat(X, x),itertools.repeat(labels, x), X, i, itertools.repeat(silVal, x))]
    p = Pool(4)
    cLabels = p.starmap(worker.comprehension, tasks)
    print(cLabels)
    p.close()
    p.join()

In [ ]:
plot_clusters("Consensus Trigram Clustering using UMAP", X, cLabels)

In [ ]:
results = evaluate_clustering('Consensus Bigram Clustering', X, labels_, cLabels)
print(results)